In [1]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

import torch

# torch.cuda.empty_cache()

In [2]:
class Identity(torch.nn.Module):
    def __init__(self):
        super(Identity, self).__init__()

    def forward(self, x):
        return x

In [3]:
import torch.nn as nn
import torch.nn.functional as F
class CNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, n_filters, filter_sizes,
                 output_dim, dropout, pad_idx):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=pad_idx)
        self.convs = nn.ModuleList([
                                    nn.Conv2d(in_channels=1,
                                              out_channels=n_filters, 
                                              kernel_size=(fs, embedding_dim))
                                    for fs in filter_sizes
                                    ])
        self.fc = nn.Linear(len(filter_sizes) * n_filters, output_dim)
        self.dropout = nn.Dropout(dropout)
        
        # self.fc = Identity()
        # self.dropout = Identity()

    def forward(self, text):
        # text = [batch size, sent len]
        embedded = self.embedding(text)

        # embedded = [batch size, sent len, emb dim]
        embedded = embedded.unsqueeze(1)

        # embedded = [batch size, 1, sent len, emb dim]
        conved = [F.relu(conv(embedded)).squeeze(3) for conv in self.convs]

        # conved_n = [batch size, n_filters, sent len - filter_sizes[n] + 1]
        pooled = [F.max_pool1d(conv, conv.shape[2]).squeeze(2) for conv in conved]

        cat = self.dropout(torch.cat(pooled, dim=1))

        return self.fc(cat)

In [265]:
class SUPER_CNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, n_filters, filter_sizes,
                 output_dim, dropout, pad_idx):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=pad_idx)
        self.convs = nn.ModuleList([
                                    nn.Conv2d(in_channels=1,
                                              out_channels=n_filters, 
                                              kernel_size=(fs, embedding_dim))
                                    for fs in filter_sizes
                                    ])

        # self.sentiment_model = CNN(vocab_size, embedding_dim, n_filters, filter_sizes,
        #          output_dim, dropout, pad_idx)
        # self.sentiment_model.load_state_dict(torch.load('CNN-sentiment.pt'))
        self.sentiment_model = torch.load('CNN-sentiment.pt')
        for param in self.sentiment_model.parameters():
          param.requires_grad = False
        self.sentiment_model.fc = Identity()
        self.sentiment_model.dropout = Identity()
        
        # self.emotion_model = CNN(vocab_size, embedding_dim, n_filters, filter_sizes,
        #          5, dropout, pad_idx)
        # self.emotion_model.load_state_dict(torch.load('CNN-emotion.pt'))
        self.emotion_model = torch.load('CNN-emotion.pt')
        for param in self.emotion_model.parameters():
          param.requires_grad = False
        self.emotion_model.fc = Identity()
        self.emotion_model.dropout = Identity()

        # self.formality_model = CNN(vocab_size, embedding_dim, n_filters, filter_sizes,
        #          output_dim, dropout, pad_idx)
        # self.formality_model.load_state_dict(torch.load('CNN-formality-binary.pt'))
        self.formality_model = torch.load('CNN-formality-binary.pt')
        for param in self.formality_model.parameters():
          param.requires_grad = False
        self.formality_model.fc = Identity()
        self.formality_model.dropout = Identity()

        # self.informativeness_model = CNN(vocab_size, embedding_dim, n_filters, filter_sizes,
        #          output_dim, dropout, pad_idx)
        # self.informativeness_model.load_state_dict(torch.load('CNN-informativeness-binary.pt'))
        self.informativeness_model = torch.load('CNN-informativeness-binary.pt')
        for param in self.informativeness_model.parameters():
          param.requires_grad = False
        self.informativeness_model.fc = Identity()
        self.informativeness_model.dropout = Identity()
        
        self.fc = nn.Linear(5 * len(filter_sizes) * n_filters, output_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, text):
        # text = [batch size, sent len]
        embedded = self.embedding(text)

        # embedded = [batch size, sent len, emb dim]
        embedded = embedded.unsqueeze(1)

        # embedded = [batch size, 1, sent len, emb dim]
        conved = [F.relu(conv(embedded)).squeeze(3) for conv in self.convs]

        # conved_n = [batch size, n_filters, sent len - filter_sizes[n] + 1]
        pooled = [F.max_pool1d(conv, conv.shape[2]).squeeze(2) for conv in conved]
        pooled = torch.cat(pooled,dim=1)

        # print("TESTING")
        o1 = self.sentiment_model(text)
        o2 = self.emotion_model(text)
        o3 = self.formality_model(text)
        o4 = self.informativeness_model(text)
        # print("TESTING 2")
        # print(pooled.type)
        # print(o1.type)
        # print(o2.type)
        cat = self.dropout(torch.cat((pooled, o1, o2, o3, o4), dim=1))
        # print(cat.shape)

        return self.fc(cat)

## Sarcasm Model Definition

In [194]:
import torch
from torchtext.legacy import data
from torchtext.legacy import datasets
import random
import numpy as np

SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

# create fields in emotion dataset
TEXT = data.Field(tokenize = 'spacy', 
                  tokenizer_language = 'en_core_web_sm',
                  batch_first = True
                  )
LABEL = data.LabelField(dtype=torch.float)

# tuples representative of tabular format
fields = [
          ('is_sarcastic', LABEL),
          ('headline', TEXT)
]

# LOAD SARCASM DATA
sarc_train, = data.TabularDataset.splits(
    path='/content',
    train='Sarcasm_Headlines_Dataset_v2.csv',
    format='csv',
    fields=fields,
    skip_header=True
)

In [266]:
MAX_VOCAB_SIZE = 25_000

sarc_train, sarc_valid, sarc_test = sarc_train.split(random_state = random.seed(SEED), split_ratio=[0.4,0.3,0.3])

TEXT.build_vocab(sarc_train, 
                 max_size = MAX_VOCAB_SIZE, 
                 vectors = "glove.6B.100d", 
                 unk_init = torch.Tensor.normal_)

LABEL.build_vocab(sarc_train)

In [267]:
BATCH_SIZE = 64

# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = torch.device('cpu')

sarc_train_iterator, sarc_valid_iterator, sarc_test_iterator = data.BucketIterator.splits(
    (sarc_train, sarc_valid, sarc_test), 
    batch_size = BATCH_SIZE, 
    device = device,
    sort=False)

In [268]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
N_FILTERS = 100
FILTER_SIZES = [3,4,5]
OUTPUT_DIM = 1
DROPOUT = 0.5
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

model = SUPER_CNN(INPUT_DIM, EMBEDDING_DIM, N_FILTERS, FILTER_SIZES, OUTPUT_DIM, DROPOUT, PAD_IDX)

In [269]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 160,801 trainable parameters


In [270]:
pretrained_embeddings = TEXT.vocab.vectors
model.embedding.weight.data.copy_(pretrained_embeddings)

# zero initial weights of <unk> and <pad>
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]

model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

In [271]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters())
criterion = nn.BCEWithLogitsLoss()
model = model.to(device)
criterion = criterion.to(device)

In [272]:
N_EPOCHS = 10
best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    start_time = time.time()
    
    train_loss, train_acc = train(model, sarc_train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, sarc_valid_iterator, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'CNN-sarcasm-extreme.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 0m 0s
	Train Loss: 0.735 | Train Acc: 57.45%
	 Val. Loss: 0.694 |  Val. Acc: 57.14%
Epoch: 02 | Epoch Time: 0m 0s
	Train Loss: 0.701 | Train Acc: 40.43%
	 Val. Loss: 0.691 |  Val. Acc: 48.57%
Epoch: 03 | Epoch Time: 0m 0s
	Train Loss: 0.663 | Train Acc: 61.70%
	 Val. Loss: 0.690 |  Val. Acc: 54.29%
Epoch: 04 | Epoch Time: 0m 0s
	Train Loss: 0.617 | Train Acc: 68.09%
	 Val. Loss: 0.690 |  Val. Acc: 62.86%
Epoch: 05 | Epoch Time: 0m 0s
	Train Loss: 0.572 | Train Acc: 78.72%
	 Val. Loss: 0.691 |  Val. Acc: 51.43%
Epoch: 06 | Epoch Time: 0m 0s
	Train Loss: 0.509 | Train Acc: 91.49%
	 Val. Loss: 0.694 |  Val. Acc: 48.57%
Epoch: 07 | Epoch Time: 0m 0s
	Train Loss: 0.488 | Train Acc: 87.23%
	 Val. Loss: 0.697 |  Val. Acc: 48.57%
Epoch: 08 | Epoch Time: 0m 0s
	Train Loss: 0.479 | Train Acc: 87.23%
	 Val. Loss: 0.697 |  Val. Acc: 48.57%
Epoch: 09 | Epoch Time: 0m 0s
	Train Loss: 0.460 | Train Acc: 91.49%
	 Val. Loss: 0.695 |  Val. Acc: 48.57%
Epoch: 10 | Epoch Time: 0m 0

In [273]:
# TESTING
model.load_state_dict(torch.load('CNN-sarcasm-extreme.pt'))

test_loss, test_acc = evaluate(model, sarc_test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 0.698 | Test Acc: 48.57%


In [21]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        # print(batch.__dict__)
        optimizer.zero_grad()
        predictions = model(batch.headline).squeeze(1)
        loss = criterion(predictions, batch.is_sarcastic)
        acc = binary_accuracy(predictions, batch.is_sarcastic)
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
        for batch in iterator:
            predictions = model(batch.headline).squeeze(1)
            loss = criterion(predictions, batch.is_sarcastic)
            acc = binary_accuracy(predictions, batch.is_sarcastic)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [5]:
from torchtext.legacy import data
from torchtext.legacy import datasets
import torch.optim as optim
import random
import numpy as np
import time

SEED = 1234


class ModelTrainer():

    def __init__(self, model_name, model_type=None, text_field_name=None,
                 label_field_name=None, IMDB=False, train_file_name=None,
                 test_file_name=None, embedding_dim=100, criterion_str="CE",
                 max_vocab_size=25000, num_filters=100, filter_sizes=[3, 4, 5],
                 dropout=0.5, output_dim=1, batch_size=64, dtype=torch.long):
        self.model_name = model_name
        self.model_type = model_type
        self.text_field_name = text_field_name
        self.label_field_name = label_field_name
        self.IMDB = IMDB
        self.train_file_name = train_file_name
        self.test_file_name = test_file_name
        self.criterion_str = criterion_str
        self.embedding_dim = embedding_dim
        self.max_vocab_size = max_vocab_size
        self.num_filters = num_filters
        self.filter_sizes = filter_sizes
        self.dropout = dropout
        self.batch_size = batch_size
        self.dtype = dtype

    def load_dataset(self):
        """ Loads the dataset using torchtext
        """
        r_seed = random.seed(SEED)
        np.random.seed(SEED)
        torch.manual_seed(SEED)
        torch.backends.cudnn.deterministic = True

        self.TEXT = data.Field(
            tokenize='spacy',
            tokenizer_language='en_core_web_sm',
            batch_first=True)
        self.LABEL = data.LabelField(dtype=self.dtype) # dtype=torch.float)
        # LOAD DATASET
        if self.IMDB:
            self.dataset, test_data = datasets.IMDB.splits(self.TEXT, self.LABEL)
            train_data, valid_data = self.dataset.split(random_state=r_seed)
        else:
            if self.text_field_name == "Sentence":
                # Formality and Informativeness don't work with dict fields
                # tuples representative of tabular format
                fields = [
                          (self.label_field_name, self.LABEL),
                          (self.text_field_name, self.TEXT)
                ]
                skip_header = True
            else:
                fields = {
                    self.label_field_name: (self.label_field_name, self.LABEL),
                    self.text_field_name: (self.text_field_name, self.TEXT)
                }
                skip_header = False
            format = self.train_file_name.split('.')[-1]
            if self.test_file_name:
                
                self.dataset, test_data = data.TabularDataset.splits(
                    path='/content',
                    train=self.train_file_name,
                    test=self.test_file_name,
                    format=format,
                    fields=fields,
                    skip_header=skip_header
                )
                train_data, valid_data = self.dataset.split(random_state=r_seed)
            else:
                self.dataset = data.TabularDataset.splits(
                    path='/content',
                    train=self.train_file_name,
                    format=format,
                    fields=fields,
                    skip_header=skip_header
                )[0]
                train_data, valid_data, test_data = self.dataset.split(
                    split_ratio=[0.4, 0.3, 0.3], random_state=r_seed)

        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        print(torch.cuda.get_device_name(0))

        self.train_iter, self.val_iter, self.test_iter = data.BucketIterator.splits(
            (train_data, valid_data, test_data),
            batch_size=self.batch_size,
            device=self.device,
            sort=False
            )

    def build_vocab(self):
        """ Builds the vocabulary of the dataset using torchtext
        """
        self.TEXT.build_vocab(
            self.dataset,
            max_size=self.max_vocab_size,
            vectors="glove.6B.100d",
            unk_init=torch.Tensor.normal_
        )
        self.LABEL.build_vocab(self.dataset)

    def init_model(self):
        """ Create CNN model with the supplied/derived parameters.
            Loads "glove.6B.100d" weights into the model's embedding layer
        """
        # Get the size of the vocabulary
        vocab_size = len(self.TEXT.vocab)
        if self.criterion_str == "CE":
          output_dim = len(self.LABEL.vocab)
        else:
          output_dim = 1
        pad_idx = self.TEXT.vocab.stoi[self.TEXT.pad_token]
        # Initialize model
        if self.model_type == 'deep':
          self.model = VDCNN(output_dim, vocab_size, pad_idx,
                             self.embedding_dim, shortcut=True)
        else:
          self.model = CNN(vocab_size, self.embedding_dim, self.num_filters,
                           self.filter_sizes, output_dim, self.dropout,
                           pad_idx)
        # Get pretrained weights from vocab
        pretrained_embeddings = self.TEXT.vocab.vectors
        self.model.embedding.weight.data.copy_(pretrained_embeddings)

        # zero initial weights of <unk> and <pad>
        unk_index = self.TEXT.vocab.stoi[self.TEXT.unk_token]

        self.model.embedding.weight.data[unk_index] = torch.zeros(self.embedding_dim)
        self.model.embedding.weight.data[pad_idx] = torch.zeros(self.embedding_dim)

    def init_optimizer(self):
        """ Initializes the optimizor (Adam) and the criterion (BCEWithLogitsLoss)
        """
        self.optimizer = optim.Adam(self.model.parameters())
        self.model = self.model.to(self.device)
        if self.criterion_str == "CE":
            self.criterion = nn.CrossEntropyLoss()
        elif self.criterion_str == "BCE":
            self.criterion = nn.BCEWithLogitsLoss()
        self.criterion = self.criterion.to(self.device)

    def train_model(self, num_epochs):
        """ Trains and validates the model (and prints the results) for the
            given number of epochs. Saves the model from the epoch which
            yeilded the lowest validation loss.

        Args:
            num_epochs (int): number of epochs to train the model
        """
        print(self.model_name)
        best_valid_loss = float('inf')

        for epoch in range(num_epochs):
            start_time = time.time()

            train_loss, train_acc = self.train_epoch()
            valid_loss, valid_acc = self.evaluate_epoch(self.val_iter)

            end_time = time.time()

            epoch_mins, epoch_secs = epoch_time(start_time, end_time)

            if valid_loss < best_valid_loss:
                best_valid_loss = valid_loss
                # torch.save(self.model.state_dict(), f'{self.model_name}.pt')
                torch.save(self.model, f'{self.model_name}.pt')

            print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
            print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
            print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

    def test_model(self):
        """ Tests the model with the highest validation loss on the test iterator
        """
        # self.model.load_state_dict(torch.load(f'{self.model_name}.pt'))
        torch.load(f'{self.model_name}.pt')
        test_loss, test_acc = self.evaluate_epoch(self.test_iter)
        print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')
        with open('model_results.txt', "a") as f:
            f.write(f'{self.model_name}\nTest Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%\n')

    def count_parameters(self):
        num_params = sum(p.numel() for p in self.model.parameters() if p.requires_grad)
        print(f'The model has {num_params:,} trainable parameters')

    def train_epoch(self):
        """ Trains the model for 1 epoch of the train iterator

        Returns:
            tuple: (loss, accuracy)
        """
        epoch_loss = 0
        epoch_acc = 0
        self.model.train()
        for batch in self.train_iter:
            # print(batch.__dict__)
            self.optimizer.zero_grad()
            predictions = self.model(getattr(batch, batch.input_fields[0]))
            # Reduce dimentionality if using binary cross entropy loss
            if self.criterion_str == "BCE":
                predictions = predictions.squeeze(1)
                acc_formula = binary_accuracy
            elif self.criterion_str == "CE":
                acc_formula = categorical_accuracy
            loss = self.criterion(predictions, getattr(batch, batch.target_fields[0]))
            acc = acc_formula(predictions, getattr(batch, batch.target_fields[0]))
            loss.backward()
            self.optimizer.step()

            epoch_loss += loss.item()
            epoch_acc += acc.item()

        return epoch_loss / len(self.train_iter), epoch_acc / len(self.train_iter)

    def evaluate_epoch(self, iterator):
        epoch_loss = 0
        epoch_acc = 0
        self.model.eval()
        with torch.no_grad():
            for batch in iterator:
                predictions = self.model(getattr(batch, batch.input_fields[0]))
                # Reduce dimentionality if using binary cross entropy loss
                if self.criterion_str == "BCE":
                    predictions = predictions.squeeze(1)
                    acc_formula = binary_accuracy
                elif self.criterion_str == "CE":
                    acc_formula = categorical_accuracy
                loss = self.criterion(predictions, getattr(batch, batch.target_fields[0]))
                acc = acc_formula(predictions, getattr(batch, batch.target_fields[0]))

                epoch_loss += loss.item()
                epoch_acc += acc.item()
        return epoch_loss / len(iterator), epoch_acc / len(iterator)
      
    def do_what_we_want(self, num_epochs=10):
        self.load_dataset()
        self.build_vocab()
        self.init_model()
        self.init_optimizer()
        self.train_model(num_epochs)
        self.test_model()


def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """
    # round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float()  # convert into float for division
    acc = correct.sum() / len(correct)
    return acc


def categorical_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """
    top_pred = preds.argmax(1, keepdim = True)
    correct = top_pred.eq(y.view_as(top_pred)).sum()
    acc = correct.float() / y.shape[0]
    return acc
  

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs


In [6]:
import gc

if __name__ == "__main__":
    # clear contents of file
    with open("model_results.txt", "w") as f:
        pass
    

    # ======================== FORMALITY BINARY MODEL ========================= #
    trainer = ModelTrainer(
        model_name="CNN-formality-binary", 
        text_field_name='Sentence', 
        label_field_name='Formality', 
        train_file_name='mturk_news_formality_BINARY.csv', 
        criterion_str='BCE',
        dtype=torch.float)
    trainer.do_what_we_want(num_epochs=8)

    
    # ======================== INFORMATIVENESS BINARY MODEL ========================= #
    trainer = ModelTrainer(
        model_name="CNN-informativeness-binary", 
        text_field_name='Sentence', 
        label_field_name='Informativeness', 
        train_file_name='mturk_news_informativeness_BINARY.csv', 
        criterion_str='BCE',
        dtype=torch.float)
    trainer.do_what_we_want(num_epochs=8)


    # ======================== EMOTION MODEL ========================= #
    trainer = ModelTrainer(
        model_name="CNN-emotion", 
        text_field_name='Text', 
        label_field_name='Emotion', 
        train_file_name='emotion_train.csv', 
        test_file_name='emotion_test.csv',
        criterion_str='CE'
    )
    trainer.do_what_we_want(num_epochs=8)
        # --------------- deep archetecture -------------------#
    # trainer = ModelTrainer(
    #     model_name="deep-CNN-emotion", 
    #     model_type='deep', 
    #     text_field_name='Text', 
    #     label_field_name='Emotion', 
    #     train_file_name='/content/emotion_train.csv', 
    #     test_file_name='/content/emotion_test.csv'
    # )
    # trainer.do_what_we_want(num_epochs=15)


    # ======================== SENTIMENT MODEL ========================= #
    trainer = ModelTrainer("CNN-sentiment", IMDB=True, criterion_str="BCE", dtype=torch.float)
    trainer.do_what_we_want(num_epochs=8)


    # ======================== SARCASM BASE MODEL ========================= #
    # trainer = ModelTrainer(
    #     model_name="CNN-sarcasm-base", 
    #     text_field_name='headline', 
    #     label_field_name='is_sarcastic', 
    #     train_file_name='Sarcasm_Headlines_Dataset_v2.csv', 
    #     criterion_str='BCE',
    #     dtype=torch.float)
    # trainer.do_what_we_want(num_epochs=8)



    # emotion_trainer = ModelTrainer(
    #     model_name="CNN-emotion", 
    #     text_field_name='Text', 
    #     label_field_name='Emotion', 
    #     train_file_name='emotion_train.csv', 
    #     test_file_name='emotion_test.csv'
    # )
    # emotion_trainer.do_what_we_want(num_epochs=8)
    # # Clear GPU memory
    # emotion_trainer = None
    # gc.collect()
    # torch.cuda.empty_cache()

    # deep_sentament_trainer = ModelTrainer("deep-CNN-sentiment", model_type='deep', IMDB=True, criterion_str="BCE")
    # deep_sentament_trainer.do_what_we_want(num_epochs=8)
    # # Clear GPU memory
    # deep_sentament_trainer = None
    # gc.collect()
    # torch.cuda.empty_cache()

.vector_cache/glove.6B.zip: 0.00B [00:00, ?B/s]

Tesla T4


.vector_cache/glove.6B.zip: 862MB [02:39, 5.40MB/s]                           
100%|█████████▉| 398077/400000 [00:15<00:00, 26980.76it/s]

CNN-formality-binary
Epoch: 01 | Epoch Time: 0m 0s
	Train Loss: 0.440 | Train Acc: 82.84%
	 Val. Loss: 0.321 |  Val. Acc: 90.85%
Epoch: 02 | Epoch Time: 0m 0s
	Train Loss: 0.285 | Train Acc: 90.21%
	 Val. Loss: 0.269 |  Val. Acc: 91.18%
Epoch: 03 | Epoch Time: 0m 0s
	Train Loss: 0.237 | Train Acc: 90.37%
	 Val. Loss: 0.248 |  Val. Acc: 91.18%
Epoch: 04 | Epoch Time: 0m 0s
	Train Loss: 0.213 | Train Acc: 91.16%
	 Val. Loss: 0.235 |  Val. Acc: 91.18%
Epoch: 05 | Epoch Time: 0m 0s
	Train Loss: 0.186 | Train Acc: 91.93%
	 Val. Loss: 0.231 |  Val. Acc: 91.41%
Epoch: 06 | Epoch Time: 0m 0s
	Train Loss: 0.169 | Train Acc: 92.65%
	 Val. Loss: 0.230 |  Val. Acc: 91.29%
Epoch: 07 | Epoch Time: 0m 0s
	Train Loss: 0.138 | Train Acc: 94.38%
	 Val. Loss: 0.229 |  Val. Acc: 91.29%
Epoch: 08 | Epoch Time: 0m 0s
	Train Loss: 0.114 | Train Acc: 95.75%
	 Val. Loss: 0.226 |  Val. Acc: 92.19%
Test Loss: 0.231 | Test Acc: 92.31%


100%|█████████▉| 398077/400000 [00:30<00:00, 26980.76it/s]

Tesla T4
CNN-informativeness-binary
Epoch: 01 | Epoch Time: 0m 0s
	Train Loss: 0.400 | Train Acc: 85.26%
	 Val. Loss: 0.241 |  Val. Acc: 94.42%
Epoch: 02 | Epoch Time: 0m 0s
	Train Loss: 0.232 | Train Acc: 93.40%
	 Val. Loss: 0.189 |  Val. Acc: 94.42%
Epoch: 03 | Epoch Time: 0m 0s
	Train Loss: 0.200 | Train Acc: 93.24%
	 Val. Loss: 0.174 |  Val. Acc: 94.42%
Epoch: 04 | Epoch Time: 0m 0s
	Train Loss: 0.183 | Train Acc: 93.16%
	 Val. Loss: 0.166 |  Val. Acc: 94.42%
Epoch: 05 | Epoch Time: 0m 0s
	Train Loss: 0.158 | Train Acc: 93.67%
	 Val. Loss: 0.160 |  Val. Acc: 94.42%
Epoch: 06 | Epoch Time: 0m 0s
	Train Loss: 0.144 | Train Acc: 93.50%
	 Val. Loss: 0.155 |  Val. Acc: 94.64%
Epoch: 07 | Epoch Time: 0m 0s
	Train Loss: 0.134 | Train Acc: 94.38%
	 Val. Loss: 0.155 |  Val. Acc: 94.42%
Epoch: 08 | Epoch Time: 0m 0s
	Train Loss: 0.111 | Train Acc: 95.14%
	 Val. Loss: 0.150 |  Val. Acc: 94.75%
Test Loss: 0.159 | Test Acc: 94.83%
Tesla T4
CNN-emotion
Epoch: 01 | Epoch Time: 0m 1s
	Train Loss: 


aclImdb_v1.tar.gz:   0%|          | 0.00/84.1M [00:00<?, ?B/s]

downloading aclImdb_v1.tar.gz



aclImdb_v1.tar.gz:   0%|          | 131k/84.1M [00:00<01:18, 1.07MB/s]
aclImdb_v1.tar.gz:   1%|          | 459k/84.1M [00:00<01:02, 1.34MB/s]
aclImdb_v1.tar.gz:   2%|▏         | 1.39M/84.1M [00:00<00:46, 1.80MB/s]
aclImdb_v1.tar.gz:   4%|▍         | 3.69M/84.1M [00:00<00:32, 2.48MB/s]
aclImdb_v1.tar.gz:   8%|▊         | 6.90M/84.1M [00:00<00:22, 3.43MB/s]
aclImdb_v1.tar.gz:  13%|█▎        | 11.3M/84.1M [00:00<00:15, 4.74MB/s]
aclImdb_v1.tar.gz:  19%|█▉        | 15.8M/84.1M [00:00<00:10, 6.48MB/s]
aclImdb_v1.tar.gz:  24%|██▍       | 20.6M/84.1M [00:00<00:07, 8.74MB/s]
aclImdb_v1.tar.gz:  30%|███       | 25.6M/84.1M [00:00<00:05, 11.6MB/s]
aclImdb_v1.tar.gz:  36%|███▋      | 30.5M/84.1M [00:01<00:03, 15.1MB/s]
aclImdb_v1.tar.gz:  43%|████▎     | 35.9M/84.1M [00:01<00:02, 19.2MB/s]
aclImdb_v1.tar.gz:  51%|█████     | 42.7M/84.1M [00:01<00:01, 24.5MB/s]
aclImdb_v1.tar.gz:  59%|█████▉    | 50.0M/84.1M [00:01<00:01, 30.6MB/s]
aclImdb_v1.tar.gz:  67%|██████▋   | 56.7M/84.1M [00:01<00:00, 36.

Tesla T4
CNN-sentiment
Epoch: 01 | Epoch Time: 0m 23s
	Train Loss: 0.654 | Train Acc: 61.26%
	 Val. Loss: 0.502 |  Val. Acc: 78.40%
Epoch: 02 | Epoch Time: 0m 23s
	Train Loss: 0.426 | Train Acc: 80.49%
	 Val. Loss: 0.353 |  Val. Acc: 84.79%
Epoch: 03 | Epoch Time: 0m 23s
	Train Loss: 0.305 | Train Acc: 87.29%
	 Val. Loss: 0.343 |  Val. Acc: 84.62%
Epoch: 04 | Epoch Time: 0m 23s
	Train Loss: 0.222 | Train Acc: 91.31%
	 Val. Loss: 0.311 |  Val. Acc: 87.18%
Epoch: 05 | Epoch Time: 0m 24s
	Train Loss: 0.157 | Train Acc: 94.32%
	 Val. Loss: 0.338 |  Val. Acc: 86.56%
Epoch: 06 | Epoch Time: 0m 24s
	Train Loss: 0.109 | Train Acc: 96.12%
	 Val. Loss: 0.337 |  Val. Acc: 87.38%
Epoch: 07 | Epoch Time: 0m 24s
	Train Loss: 0.072 | Train Acc: 97.76%
	 Val. Loss: 0.366 |  Val. Acc: 87.16%
Epoch: 08 | Epoch Time: 0m 24s
	Train Loss: 0.052 | Train Acc: 98.32%
	 Val. Loss: 0.397 |  Val. Acc: 87.28%
Test Loss: 0.447 | Test Acc: 85.37%
